# Predicciones con RandomForest

A continuación voy a aplicar mi modelo para predecir el tipo de sonido en base a un dataset nuevo de sonidos.

Empiezo importando las librerías que voy a necesitar.

In [12]:
import numpy as np
import pandas as pd
import pandas.io

import re

import matplotlib.pyplot as plt

import librosa.display
import librosa

from glob import glob

import ffmpeg

import os

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import pickle

Empiezo importando el modelo que he guardado anteriormente, para trabajar con él.

In [4]:
filename = 'finalized_model_fourier.sav'

loaded_model = pickle.load(open(filename, 'rb'))

Continúo descomponiendo los audios en segmentos de 2 segundos, y sacando todas sus features.

In [ ]:
def decompose_files(data_dir, audio_files):
    for j in range(len(audio_files)):
        myaudio = AudioSegment.from_file(data_dir + '{}'.format(os.listdir(data_dir)[j])) 
        chunk_length_ms = 2000
        chunks = make_chunks(myaudio, chunk_length_ms)

        for i, chunk in enumerate(chunks):
            chunk_name = "{}{}.wav".format(os.listdir(data_dir)[j], i)
            #print("exporting", chunk_name)
            chunk.export('./audios/predicciones_fourier_32/{}'.format(chunk_name), format="mp3")
            
decompose_files('./audios/predicciones/', glob('./audios/predicciones' + '/*'))

In [ ]:
def get_features(data_dir, audio_files):
    momentos = {'ducha': 'Ducha', 'cena': 'Cena', 'washing': 'Lavadora', 'vacuum': 'Aspiradora', 
                'shaver': 'Afeitadora', 'hairdryer': 'Secador_pelo', 'airconditioner': 'Aire_acondicionado', 
                'cellphone': 'Telefono', 'comp': 'Tecleo', 'silence': 'Silencio', 'dryer': 'Secadora', 
                'blender': 'Licuadora', 'doorbell': 'Timbre', 'alarm': 'Alarma', 'faucet': 'Grifo', 
                'microwave': 'Microondas'}
    features = []
    for i in range(len(audio_files)):
        y, sr = librosa.load(audio_files[i], sr=8000, mono=True)
        name = os.listdir(data_dir)[i]
        pattern = "[._][\w]+"
        name = re.sub(pattern, '', name)
        if name in momentos.keys():
            momento = momentos.get(name)
        else:
            momento = 'Otro'
        mfcc = np.ndarray.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20))
        scem = np.ndarray.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
        scom = np.ndarray.mean(librosa.feature.spectral_contrast(S=np.abs(librosa.stft(y)), sr=sr, n_bands=4))
        srom = np.ndarray.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
        sbwm = np.ndarray.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
        tempo = librosa.beat.tempo(onset_envelope=librosa.onset.onset_strength(y=y, sr=sr, hop_length=512), sr=sr, hop_length=512)[0]
        rmse = np.ndarray.mean(librosa.feature.rms(y=y))
        D = np.abs(np.fft.fft(y, n=32)[:8000*2 // 2])
        features.append([mfcc, scem, scom, srom, sbwm, tempo, rmse, 
                         D[0], D[1], D[2], D[3], D[4], D[5], 
                         D[6], D[7], D[8], D[9], D[10], D[11], 
                         D[12], D[13], D[14], D[15], D[16], D[17], 
                         D[18], D[19], D[20], D[21], D[22], D[23],
                         D[24], D[25], D[26], D[27], D[28], D[29],
                         D[30], D[31], momento])
    return pd.DataFrame(features, columns=['mfcc', 'scem','scom', 'srom','sbwm', 'tempo', 'rmse', 
                                           'Fourier1', 'Fourier2', 'Fourier3', 'Fourier4', 'Fourier5', 
                                           'Fourier6', 'Fourier7', 'Fourier8', 'Fourier9', 'Fourier10',
                                           'Fourier11', 'Fourier12', 'Fourier13', 'Fourier14', 'Fourier15',
                                           'Fourier16', 'Fourier17', 'Fourier18', 'Fourier19', 'Fourier20',
                                           'Fourier21', 'Fourier22', 'Fourier23', 'Fourier24', 'Fourier25',
                                           'Fourier26', 'Fourier27', 'Fourier28', 'Fourier29', 'Fourier30',
                                           'Fourier31', 'Fourier32', 'momento'])

prueba = get_features('./audios/predicciones_fourier_32/', glob('./audios/predicciones_fourier_32' + '/*.wav'))

In [ ]:
prueba.to_csv('data_fourier_aux.csv', index=False)

In [5]:
prueba = pd.read_csv('data_fourier_aux.csv')
prueba.head()

,mfcc,scem,scom,srom,sbwm,tempo,rmse,Fourier1,Fourier2,Fourier3,...,Fourier24,Fourier25,Fourier26,Fourier27,Fourier28,Fourier29,Fourier30,Fourier31,Fourier32,momento
0,-14.789252,1307.712330,22.329178,2490.112305,941.869915,117.187500,0.029125,0.105356,0.059353,0.475078,...,0.091964,0.040705,0.076092,0.114835,0.089264,0.368764,0.326332,0.475078,0.059353,Lavadora
1,-2.996813,1181.177661,33.464148,3242.187500,1190.281966,120.000000,0.085731,0.121791,0.494262,1.742387,...,0.066197,0.159545,0.212939,0.129471,0.306812,0.260177,0.868978,1.742387,0.494262,Microondas
2,-2.807277,1409.495596,23.275813,2779.785156,1055.763228,133.928571,0.096703,0.776670,0.482565,0.618999,...,0.043657,0.057000,0.089782,0.095835,0.067161,0.248390,0.591389,0.618999,0.482565,Aspiradora
3,-16.530776,707.460976,23.874835,747.314453,440.637551,133.928571,0.017683,0.101806,0.120892,0.529542,...,0.024370,0.029982,0.034488,0.045352,0.056777,0.091805,0.242167,0.529542,0.120892,Timbre
4,-4.882842,1925.234669,19.538132,3061.645508,968.023202,133.928571,0.038148,0.096550,0.043815,0.224315,...,0.200003,0.290234,0.287588,0.155598,0.354692,0.042967,0.173730,0.224315,0.043815,Ducha


En este paso intermedio, dado que hay una pequeña cantidad de audios cuyas features son todas cero, los elimino para no ensuciar el modelo.

In [6]:
prueba = prueba[prueba['Fourier24'] != 0].reset_index(drop=True)
prueba.head()

,mfcc,scem,scom,srom,sbwm,tempo,rmse,Fourier1,Fourier2,Fourier3,...,Fourier24,Fourier25,Fourier26,Fourier27,Fourier28,Fourier29,Fourier30,Fourier31,Fourier32,momento
0,-14.789252,1307.712330,22.329178,2490.112305,941.869915,117.187500,0.029125,0.105356,0.059353,0.475078,...,0.091964,0.040705,0.076092,0.114835,0.089264,0.368764,0.326332,0.475078,0.059353,Lavadora
1,-2.996813,1181.177661,33.464148,3242.187500,1190.281966,120.000000,0.085731,0.121791,0.494262,1.742387,...,0.066197,0.159545,0.212939,0.129471,0.306812,0.260177,0.868978,1.742387,0.494262,Microondas
2,-2.807277,1409.495596,23.275813,2779.785156,1055.763228,133.928571,0.096703,0.776670,0.482565,0.618999,...,0.043657,0.057000,0.089782,0.095835,0.067161,0.248390,0.591389,0.618999,0.482565,Aspiradora
3,-16.530776,707.460976,23.874835,747.314453,440.637551,133.928571,0.017683,0.101806,0.120892,0.529542,...,0.024370,0.029982,0.034488,0.045352,0.056777,0.091805,0.242167,0.529542,0.120892,Timbre
4,-4.882842,1925.234669,19.538132,3061.645508,968.023202,133.928571,0.038148,0.096550,0.043815,0.224315,...,0.200003,0.290234,0.287588,0.155598,0.354692,0.042967,0.173730,0.224315,0.043815,Ducha


In [7]:
classes = ['Afeitadora', 'Aire_acondicionado', 'Alarma', 'Aspiradora', 'Cena', 'Ducha', 'Grifo', 'Lavadora', 
           'Licuadora', 'Microondas', 'Secador_pelo', 'Secadora', 'Silencio', 'Tecleo', 'Telefono', 'Timbre']

X_new = prueba.loc[:, prueba.columns != 'momento']
y_new = loaded_model.predict(X_new)
y_new2 = loaded_model.predict_proba(X_new)

reconocidos = 0
aciertos = 0
for i in range(len(X_new)):
    predicted_list = y_new[i].tolist()
    predicted_class = classes[y_new2[i].tolist().index(max(y_new2[i]))]
    predicted_probability = y_new2[i].tolist()[y_new2[i].tolist().index(max(y_new2[i]))]
    if max(y_new2[i]) > 0:
        if predicted_class == prueba.momento[i]:
            #print(str(i) + '-  {}: {:.2f}% ----> {}'.format(predicted_class, predicted_probability*100, prueba.momento[i]))
            aciertos += 1
            reconocidos += 1
        else:
            #print(str(i) + '-  {}: {:.2f}% ----> {}'.format(predicted_class, predicted_probability*100, prueba.momento[i]))
            reconocidos += 1


print('Porcentaje de sonidos reconocidos: {:.2f}%.'.format(reconocidos/(len(X_new))*100))
print('Porcentaje de aciertos: {:.2f}%.'.format(aciertos/(len(X_new))*100))
print('Porcentaje de aciertos una vez reconoce el sonido: {:.2f}%.'.format(aciertos/reconocidos*100))

Porcentaje de sonidos reconocidos: 100.00%.
Porcentaje de aciertos: 52.65%.
Porcentaje de aciertos una vez reconoce el sonido: 52.65%.


# Predicciones con el modelo PCA de 20 features

Hago lo mismo que antes pero con el nuevo modelo sobre el que hemos aplicado el PCA.

In [27]:
filename = 'finalized_model_fourier_32_PCA.sav'

new_model = pickle.load(open(filename, 'rb'))

In [28]:
features = ['mfcc', 'scem','scom', 'srom','sbwm', 'tempo', 'rmse', 
            'Fourier1', 'Fourier2', 'Fourier3', 'Fourier4', 'Fourier5', 
            'Fourier6', 'Fourier7', 'Fourier8', 'Fourier9', 'Fourier10',
            'Fourier11', 'Fourier12', 'Fourier13', 'Fourier14', 'Fourier15',
            'Fourier16', 'Fourier17', 'Fourier18', 'Fourier19', 'Fourier20',
            'Fourier21', 'Fourier22', 'Fourier23', 'Fourier24', 'Fourier25',
            'Fourier26', 'Fourier27', 'Fourier28', 'Fourier29', 'Fourier30',
            'Fourier31', 'Fourier32']

x = prueba.loc[:, features].values
y = prueba.loc[:,['momento']].values

x = StandardScaler().fit_transform(x)

In [29]:
pca = PCA(n_components=20)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents, 
                           columns = ['principal component 1',
                                      'principal component 2',
                                      'principal component 3',
                                      'principal component 4',
                                      'principal component 5',
                                      'principal component 6',
                                      'principal component 7',
                                      'principal component 8',
                                      'principal component 9',
                                      'principal component 10',
                                      'principal component 11',
                                      'principal component 12',
                                      'principal component 13',
                                      'principal component 14',
                                      'principal component 15',
                                      'principal component 16',
                                      'principal component 17',
                                      'principal component 18',
                                      'principal component 19',
                                      'principal component 20'])

principalDf.head()

,principal component 1,principal component 2,principal component 3,principal component 4,principal component 5,principal component 6,principal component 7,principal component 8,principal component 9,principal component 10,principal component 11,principal component 12,principal component 13,principal component 14,principal component 15,principal component 16,principal component 17,principal component 18,principal component 19,principal component 20
0,-2.365622,0.266966,-1.244869,-0.610827,-0.326512,-0.393226,0.017619,0.023030,0.369892,0.543561,0.253198,-0.159291,-0.231899,-0.603605,0.008237,0.006586,0.263810,0.020484,-0.168861,0.338656
1,2.852451,0.285004,-1.821688,0.415679,0.690888,4.416931,1.731095,2.440102,-2.651443,1.446260,0.797143,1.587254,0.871853,0.012771,-0.920716,-1.198910,0.916135,0.666444,-0.039656,-0.172282
2,-0.016743,0.419959,-0.200263,-0.009293,-0.270463,1.005343,0.591799,0.650574,-1.582151,-0.421844,1.211510,0.195842,0.081110,0.532983,0.826914,-0.421365,0.519174,0.957072,0.282322,0.145431
3,-3.176075,-1.491012,-0.043447,0.304897,-0.596193,-0.334017,2.042961,0.288047,1.014553,0.580449,-1.137568,-0.003625,-0.745002,-0.130643,-0.329169,-0.125806,-0.032272,0.128384,-0.468283,0.763622
4,-0.358093,1.671443,0.376462,-0.271902,0.119153,-0.383504,-0.506464,-0.787202,-0.278277,-0.506000,-0.413457,0.825775,0.071694,0.572023,-0.074908,0.327389,-0.231408,-0.354573,0.134275,-0.770285


In [30]:
principalDf.reset_index(drop=True, inplace=True)

aux = prueba[['momento']]
aux.reset_index(drop=True, inplace=True)

finalDf = pd.concat([principalDf, aux], axis = 1)
finalDf.head()

,principal component 1,principal component 2,principal component 3,principal component 4,principal component 5,principal component 6,principal component 7,principal component 8,principal component 9,principal component 10,...,principal component 12,principal component 13,principal component 14,principal component 15,principal component 16,principal component 17,principal component 18,principal component 19,principal component 20,momento
0,-2.365622,0.266966,-1.244869,-0.610827,-0.326512,-0.393226,0.017619,0.023030,0.369892,0.543561,...,-0.159291,-0.231899,-0.603605,0.008237,0.006586,0.263810,0.020484,-0.168861,0.338656,Lavadora
1,2.852451,0.285004,-1.821688,0.415679,0.690888,4.416931,1.731095,2.440102,-2.651443,1.446260,...,1.587254,0.871853,0.012771,-0.920716,-1.198910,0.916135,0.666444,-0.039656,-0.172282,Microondas
2,-0.016743,0.419959,-0.200263,-0.009293,-0.270463,1.005343,0.591799,0.650574,-1.582151,-0.421844,...,0.195842,0.081110,0.532983,0.826914,-0.421365,0.519174,0.957072,0.282322,0.145431,Aspiradora
3,-3.176075,-1.491012,-0.043447,0.304897,-0.596193,-0.334017,2.042961,0.288047,1.014553,0.580449,...,-0.003625,-0.745002,-0.130643,-0.329169,-0.125806,-0.032272,0.128384,-0.468283,0.763622,Timbre
4,-0.358093,1.671443,0.376462,-0.271902,0.119153,-0.383504,-0.506464,-0.787202,-0.278277,-0.506000,...,0.825775,0.071694,0.572023,-0.074908,0.327389,-0.231408,-0.354573,0.134275,-0.770285,Ducha


In [31]:
classes = ['Afeitadora', 'Aire_acondicionado', 'Alarma', 'Aspiradora', 'Cena', 'Ducha', 'Grifo', 'Lavadora', 
           'Licuadora', 'Microondas', 'Secador_pelo', 'Secadora', 'Silencio', 'Tecleo', 'Telefono', 'Timbre']

X_new = finalDf.loc[:, finalDf.columns != 'momento']
y_new = new_model.predict(X_new)
y_new2 = new_model.predict_proba(X_new)

reconocidos = 0
aciertos = 0
for i in range(len(X_new)):
    predicted_list = y_new[i].tolist()
    predicted_class = classes[y_new2[i].tolist().index(max(y_new2[i]))]
    predicted_probability = y_new2[i].tolist()[y_new2[i].tolist().index(max(y_new2[i]))]
    if max(y_new2[i]) > 0:
        if predicted_class == finalDf.momento[i]:
            #print(str(i) + '-  {}: {:.2f}% ----> {}'.format(predicted_class, predicted_probability*100, finalDf.momento[i]))
            aciertos += 1
            reconocidos += 1
        else:
            #print(str(i) + '-  {}: {:.2f}% ----> {}'.format(predicted_class, predicted_probability*100, finalDf.momento[i]))
            reconocidos += 1


print('Porcentaje de sonidos reconocidos: {:.2f}%.'.format(reconocidos/(len(X_new))*100))
print('Porcentaje de aciertos: {:.2f}%.'.format(aciertos/(len(X_new))*100))
print('Porcentaje de aciertos una vez reconoce el sonido: {:.2f}%.'.format(aciertos/reconocidos*100))

Porcentaje de sonidos reconocidos: 100.00%.
Porcentaje de aciertos: 5.96%.
Porcentaje de aciertos una vez reconoce el sonido: 5.96%.


# Conclusiones

Los resultados me sorprenden, dado que me esperaba que el modelo con el PCA me diese mejores resultados, dado que tenía un mejor score. Creo que podría deberse a que aunque son dos datasets de audios del mismo tipo, al ser sonidos diferentes, al aplicar el PCA de forma independiente me da unas relaciones y unos coeficientes diferentes.

Aparte de eso, el modelo es capaz de predecirme un 52% de los audios. Mi dataset consta de 4 tipos diferentes de sonidos para cada clase, por lo que no es un dataset muy rico, y mi dataset a predecir consta de 3 tipo diferentes para cada clase. Por lo que en base a esta escasa variedad de variables, los resultados considero que no son nada desdeñables.

Como próximo paso, convendría enriquecer mi dataset.